# 1.19b4: Identify Live vs Dead Tokens

**Goal:** Determine *exactly* which tokens will be trained by tokenizing the actual model training corpus.

## Why This Matters

In **1.19b2** we *predicted* that ~1,272 Thai tokens would be "dead" (never appear during training).

But why predict when we can **measure**?

By tokenizing our actual English-only training corpus with the Flannel tokenizer, we can determine with **perfect certainty**:
- Which tokens appear ("live" - will receive gradient updates)
- Which tokens never appear ("dead" - stay frozen at initialization)

## This Notebook

1. Load the Flannel tokenizer
2. Load the model training corpus (English-only, from 1.18a)
3. Tokenize the entire corpus
4. Identify which tokens appear vs don't appear
5. Create masks and indices for live/dead tokens
6. Save for use in training and analysis

## Why Masks?

Saving boolean masks lets us:
- Filter embeddings during analysis: `W[live_mask]` or `W[dead_mask]`
- Count live vs dead: `live_mask.sum()` vs `dead_mask.sum()`
- Validate experiments: "Did exactly the dead tokens stay frozen?"

If we change the corpus or tokenizer, just rerun this notebook to update masks!

## Parameters

In [1]:
# Input files
TOKENIZER_PATH = "../data/flannel_tokenizer_chars.json"
TRAINING_CORPUS_PATH = "../data/flannel_model_corpus.txt"  # From 1.18a

# Output
OUTPUT_PATH = "../tensors/Flannel/live_dead_tokens.safetensors"

print("✓ Parameters set")

✓ Parameters set


## Imports

In [2]:
from tokenizers import Tokenizer
from pathlib import Path
from collections import Counter
import torch
from safetensors.torch import save_file

print("✓ Imports complete")

✓ Imports complete


## Load Tokenizer

In [3]:
print(f"Loading tokenizer from {TOKENIZER_PATH}...\n")

tokenizer = Tokenizer.from_file(str(TOKENIZER_PATH))
vocab = tokenizer.get_vocab()
vocab_size = len(vocab)

print(f"✓ Loaded tokenizer")
print(f"  Vocabulary size: {vocab_size:,} tokens")

Loading tokenizer from ../data/flannel_tokenizer_chars.json...

✓ Loaded tokenizer
  Vocabulary size: 10,000 tokens


## Load Training Corpus

In [4]:
corpus_path = Path(TRAINING_CORPUS_PATH)

if not corpus_path.exists():
    raise FileNotFoundError(f"Training corpus not found at {TRAINING_CORPUS_PATH}. Run 1.18a first.")

print(f"\nLoading training corpus from {TRAINING_CORPUS_PATH}...\n")

with open(corpus_path, 'r', encoding='utf-8') as f:
    corpus_text = f.read()

corpus_bytes = len(corpus_text.encode('utf-8'))
corpus_mb = corpus_bytes / (1024 * 1024)
corpus_chars = len(corpus_text)

print(f"✓ Loaded training corpus")
print(f"  Size: {corpus_bytes:,} bytes ({corpus_mb:.2f} MB)")
print(f"  Characters: {corpus_chars:,}")


Loading training corpus from ../data/flannel_model_corpus.txt...

✓ Loaded training corpus
  Size: 5,257,019 bytes (5.01 MB)
  Characters: 5,225,690


## Tokenize Corpus

This is the key step: tokenize the entire training corpus to see which tokens actually appear.

In [5]:
print(f"\nTokenizing corpus...\n")

# Tokenize
encoding = tokenizer.encode(corpus_text)
token_ids = encoding.ids

print(f"✓ Tokenization complete")
print(f"  Total tokens: {len(token_ids):,}")
print(f"  Unique tokens: {len(set(token_ids)):,}")
print(f"  Tokens per character: {len(token_ids) / corpus_chars:.3f}")


Tokenizing corpus...

✓ Tokenization complete
  Total tokens: 1,371,328
  Unique tokens: 6,301
  Tokens per character: 0.262


## Identify Live vs Dead Tokens

In [6]:
print(f"\nIdentifying live vs dead tokens...\n")

# Get set of tokens that appear in corpus
tokens_in_corpus = set(token_ids)

# Count occurrences
token_counts = Counter(token_ids)

# Classify every token in vocabulary
live_token_ids = sorted(tokens_in_corpus)
dead_token_ids = sorted(set(range(vocab_size)) - tokens_in_corpus)

num_live = len(live_token_ids)
num_dead = len(dead_token_ids)

print(f"✓ Classification complete")
print(f"  Live tokens: {num_live:,} ({100*num_live/vocab_size:.2f}%)")
print(f"  Dead tokens: {num_dead:,} ({100*num_dead/vocab_size:.2f}%)")
print()

# Show token frequency distribution
print(f"Token frequency statistics:")
counts = list(token_counts.values())
print(f"  Min occurrences: {min(counts):,}")
print(f"  Max occurrences: {max(counts):,}")
print(f"  Mean occurrences: {sum(counts)/len(counts):.1f}")
print(f"  Median occurrences: {sorted(counts)[len(counts)//2]:,}")


Identifying live vs dead tokens...

✓ Classification complete
  Live tokens: 6,301 (63.01%)
  Dead tokens: 3,699 (36.99%)

Token frequency statistics:
  Min occurrences: 1
  Max occurrences: 45,630
  Mean occurrences: 217.6
  Median occurrences: 68


## Create Masks and Indices

In [7]:
print(f"\nCreating masks and indices...\n")

# Create boolean masks
live_mask = torch.zeros(vocab_size, dtype=torch.bool)
live_mask[live_token_ids] = True

dead_mask = ~live_mask

# Create index tensors (for advanced indexing)
live_indices = torch.tensor(live_token_ids, dtype=torch.long)
dead_indices = torch.tensor(dead_token_ids, dtype=torch.long)

# Create count tensor (how many times each token appears)
token_occurrence_counts = torch.zeros(vocab_size, dtype=torch.long)
for token_id, count in token_counts.items():
    token_occurrence_counts[token_id] = count

print(f"✓ Created masks and indices")
print(f"  live_mask: {live_mask.shape} bool tensor")
print(f"  dead_mask: {dead_mask.shape} bool tensor")
print(f"  live_indices: {live_indices.shape} long tensor")
print(f"  dead_indices: {dead_indices.shape} long tensor")
print(f"  token_occurrence_counts: {token_occurrence_counts.shape} long tensor")
print()

# Verify
print(f"Verification:")
print(f"  live_mask.sum() = {live_mask.sum().item():,} (expected {num_live:,})")
print(f"  dead_mask.sum() = {dead_mask.sum().item():,} (expected {num_dead:,})")
print(f"  live_indices.numel() = {live_indices.numel():,}")
print(f"  dead_indices.numel() = {dead_indices.numel():,}")
print(f"  token_occurrence_counts.sum() = {token_occurrence_counts.sum().item():,} (total tokens)")


Creating masks and indices...

✓ Created masks and indices
  live_mask: torch.Size([10000]) bool tensor
  dead_mask: torch.Size([10000]) bool tensor
  live_indices: torch.Size([6301]) long tensor
  dead_indices: torch.Size([3699]) long tensor
  token_occurrence_counts: torch.Size([10000]) long tensor

Verification:
  live_mask.sum() = 6,301 (expected 6,301)
  dead_mask.sum() = 3,699 (expected 3,699)
  live_indices.numel() = 6,301
  dead_indices.numel() = 3,699
  token_occurrence_counts.sum() = 1,371,328 (total tokens)


## Analyze Dead Tokens by Category

Which categories from 1.19b2 are dead?

In [8]:
import json

# Load demographics from 1.19b2
demographics_path = "../data/flannel_token_demographics.json"
if Path(demographics_path).exists():
    print(f"\nAnalyzing dead tokens by category...\n")
    
    with open(demographics_path, 'r', encoding='utf-8') as f:
        demographics = json.load(f)
    
    # Count dead tokens by category
    dead_by_category = Counter()
    for token_id in dead_token_ids:
        token_id_str = str(token_id)
        if token_id_str in demographics['token_classifications']:
            category = demographics['token_classifications'][token_id_str]['category']
            dead_by_category[category] += 1
    
    print(f"Dead tokens by category:")
    for category in ['english', 'thai', 'mixed', 'numeric', 'whitespace', 'punctuation', 'special', 'other']:
        count = dead_by_category[category]
        if count > 0:
            pct = 100 * count / num_dead
            print(f"  {category.capitalize():12s}: {count:5,} ({pct:5.1f}% of dead tokens)")
    
    # Compare to prediction
    print()
    predicted_dead = demographics['dead_token_estimate']
    print(f"Prediction vs Reality:")
    print(f"  Predicted dead (from 1.19b2): {predicted_dead:,} tokens")
    print(f"  Actual dead: {num_dead:,} tokens")
    print(f"  Difference: {abs(num_dead - predicted_dead):,} tokens")
    
    if abs(num_dead - predicted_dead) < 100:
        print(f"  ✓ Excellent prediction! (<100 token error)")
    elif abs(num_dead - predicted_dead) < 500:
        print(f"  ✓ Good prediction (<500 token error)")
    else:
        print(f"  ⚠ Prediction was off by {abs(num_dead - predicted_dead):,} tokens")
else:
    print(f"\n(Demographics data not found, skipping category analysis)")


Analyzing dead tokens by category...

Dead tokens by category:
  English     :    35 (  0.9% of dead tokens)
  Thai        : 1,272 ( 34.4% of dead tokens)
  Numeric     :     1 (  0.0% of dead tokens)
  Whitespace  :     3 (  0.1% of dead tokens)
  Punctuation :     3 (  0.1% of dead tokens)
  Special     :     1 (  0.0% of dead tokens)
  Other       : 2,384 ( 64.4% of dead tokens)

Prediction vs Reality:
  Predicted dead (from 1.19b2): 1,272 tokens
  Actual dead: 3,699 tokens
  Difference: 2,427 tokens
  ⚠ Prediction was off by 2,427 tokens


## Show Examples

In [9]:
print(f"\n{'='*70}")
print(f"EXAMPLES")
print(f"{'='*70}\n")

# Reverse vocab for lookups
id_to_token = {v: k for k, v in vocab.items()}

print(f"Most frequent live tokens (top 20):")
most_common = token_counts.most_common(20)
for token_id, count in most_common:
    token_str = id_to_token[token_id]
    print(f"  {token_id:5d}: {repr(token_str):20s} - {count:,} occurrences")
print()

print(f"Least frequent live tokens (bottom 20):")
least_common = sorted(token_counts.items(), key=lambda x: x[1])[:20]
for token_id, count in least_common:
    token_str = id_to_token[token_id]
    print(f"  {token_id:5d}: {repr(token_str):20s} - {count:,} occurrences")
print()

print(f"Sample dead tokens (first 20):")
for token_id in dead_token_ids[:20]:
    token_str = id_to_token[token_id]
    print(f"  {token_id:5d}: {repr(token_str)}")

if len(dead_token_ids) > 20:
    print(f"  ... and {len(dead_token_ids) - 20:,} more")


EXAMPLES

Most frequent live tokens (top 20):
     16: '.'                  - 45,630 occurrences
     14: ','                  - 45,066 occurrences
   2806: 'the'                - 40,579 occurrences
   2818: 'and'                - 25,487 occurrences
   2815: 'to'                 - 23,515 occurrences
   2822: 'of'                 - 22,989 occurrences
     67: 'a'                  - 20,791 occurrences
   2802: 'in'                 - 17,296 occurrences
     85: 's'                  - 13,280 occurrences
     15: '-'                  - 13,160 occurrences
   2814: 'is'                 - 10,849 occurrences
   2836: 'for'                - 9,035 occurrences
   2805: 'on'                 - 8,278 occurrences
   2850: 'that'               - 8,255 occurrences
   2816: 'ing'                - 6,992 occurrences
     43: 'I'                  - 6,917 occurrences
    896: '’'                  - 6,742 occurrences
   2820: 'it'                 - 6,517 occurrences
   2817: 'ed'                 - 6,508 occu

## Save Masks and Indices

In [10]:
print(f"\nSaving masks to {OUTPUT_PATH}...\n")

# Prepare data dictionary
save_data = {
    'live_mask': live_mask,
    'dead_mask': dead_mask,
    'live_indices': live_indices,
    'dead_indices': dead_indices,
    'token_occurrence_counts': token_occurrence_counts,
}

# Ensure directory exists
Path(OUTPUT_PATH).parent.mkdir(parents=True, exist_ok=True)

# Save
save_file(save_data, str(OUTPUT_PATH))

# Verify
output_path = Path(OUTPUT_PATH)
if output_path.exists():
    output_kb = output_path.stat().st_size / 1024
    print(f"✓ Saved masks and indices")
    print(f"  Path: {OUTPUT_PATH}")
    print(f"  Size: {output_kb:.1f} KB")
else:
    raise FileNotFoundError(f"Failed to save to {OUTPUT_PATH}")


Saving masks to ../tensors/Flannel/live_dead_tokens.safetensors...

✓ Saved masks and indices
  Path: ../tensors/Flannel/live_dead_tokens.safetensors
  Size: 176.2 KB


## Usage Examples

Show how to use these masks in analysis.

In [11]:
print(f"\n{'='*70}")
print(f"USAGE EXAMPLES")
print(f"{'='*70}\n")

print(f"How to use these masks in analysis:\n")

print(f"1. Filter W matrix to only live tokens:")
print(f"   from safetensors.torch import load_file")
print(f"   masks = load_file('{OUTPUT_PATH}')")
print(f"   W_live = W[masks['live_mask']]")
print(f"   # Shape: ({num_live}, hidden_dim)\n")

print(f"2. Filter to only dead tokens:")
print(f"   W_dead = W[masks['dead_mask']]")
print(f"   # Shape: ({num_dead}, hidden_dim)\n")

print(f"3. Get specific live tokens by index:")
print(f"   W_specific = W[masks['live_indices'][:100]]")
print(f"   # First 100 live tokens\n")

print(f"4. Check if a token is live:")
print(f"   token_id = 42")
print(f"   is_live = masks['live_mask'][token_id].item()")
print(f"   # True or False\n")

print(f"5. Count tokens by category:")
print(f"   num_live = masks['live_mask'].sum().item()")
print(f"   num_dead = masks['dead_mask'].sum().item()")
print(f"   # {num_live:,} live, {num_dead:,} dead\n")

print(f"6. Weighted analysis by token frequency:")
print(f"   counts = masks['token_occurrence_counts']")
print(f"   live_counts = counts[masks['live_mask']]")
print(f"   # Analyze tokens weighted by how often they appear")


USAGE EXAMPLES

How to use these masks in analysis:

1. Filter W matrix to only live tokens:
   from safetensors.torch import load_file
   masks = load_file('../tensors/Flannel/live_dead_tokens.safetensors')
   W_live = W[masks['live_mask']]
   # Shape: (6301, hidden_dim)

2. Filter to only dead tokens:
   W_dead = W[masks['dead_mask']]
   # Shape: (3699, hidden_dim)

3. Get specific live tokens by index:
   W_specific = W[masks['live_indices'][:100]]
   # First 100 live tokens

4. Check if a token is live:
   token_id = 42
   is_live = masks['live_mask'][token_id].item()
   # True or False

5. Count tokens by category:
   num_live = masks['live_mask'].sum().item()
   num_dead = masks['dead_mask'].sum().item()
   # 6,301 live, 3,699 dead

6. Weighted analysis by token frequency:
   counts = masks['token_occurrence_counts']
   live_counts = counts[masks['live_mask']]
   # Analyze tokens weighted by how often they appear


## Summary

In [12]:
print(f"\n{'='*70}")
print(f"LIVE/DEAD TOKEN ANALYSIS COMPLETE")
print(f"{'='*70}\n")

print(f"Training corpus:")
print(f"  Path: {TRAINING_CORPUS_PATH}")
print(f"  Size: {corpus_mb:.2f} MB")
print(f"  Total tokens after tokenization: {len(token_ids):,}\n")

print(f"Tokenizer:")
print(f"  Path: {TOKENIZER_PATH}")
print(f"  Vocabulary: {vocab_size:,} tokens\n")

print(f"Results (EMPIRICAL, not predicted):")
print(f"  Live tokens: {num_live:,} ({100*num_live/vocab_size:.1f}%)")
print(f"  Dead tokens: {num_dead:,} ({100*num_dead/vocab_size:.1f}%)\n")

print(f"Output:")
print(f"  Masks and indices: {OUTPUT_PATH}\n")

print(f"What this means:")
print(f"  ✓ {num_live:,} tokens WILL receive gradient updates during training")
print(f"  ✓ {num_dead:,} tokens will NEVER appear in training")
print(f"  ✓ Dead tokens should stay frozen at initialization")
print(f"  ✓ Perfect validation: count frozen tokens after training\n")

print(f"Next steps:")
print(f"  → Train Flannel 1 (notebook 1.20a)")
print(f"  → Use these masks to analyze live vs dead token behavior")
print(f"  → Validate: did dead tokens stay frozen?")
print(f"  → If corpus or tokenizer changes, rerun this notebook")
print()
print(f"{'='*70}")


LIVE/DEAD TOKEN ANALYSIS COMPLETE

Training corpus:
  Path: ../data/flannel_model_corpus.txt
  Size: 5.01 MB
  Total tokens after tokenization: 1,371,328

Tokenizer:
  Path: ../data/flannel_tokenizer_chars.json
  Vocabulary: 10,000 tokens

Results (EMPIRICAL, not predicted):
  Live tokens: 6,301 (63.0%)
  Dead tokens: 3,699 (37.0%)

Output:
  Masks and indices: ../tensors/Flannel/live_dead_tokens.safetensors

What this means:
  ✓ 6,301 tokens WILL receive gradient updates during training
  ✓ 3,699 tokens will NEVER appear in training
  ✓ Dead tokens should stay frozen at initialization
  ✓ Perfect validation: count frozen tokens after training

Next steps:
  → Train Flannel 1 (notebook 1.20a)
  → Use these masks to analyze live vs dead token behavior
  → Validate: did dead tokens stay frozen?
  → If corpus or tokenizer changes, rerun this notebook

